# Hands-on: Parallel Computing Applied to Industry

Welcome to _Hands-on_. In this short course you will learn several techniques for scaling computation on industrial applications, with an emphasis on [OPENMP, OPENACC, CUDA] which allows for elegant parallelization applications codes and has been proven to scale very well on supercomputational systems.

## The Coding Environment

For your work today, you have access to several computational resources in the cloud. Run the following cell to see the features available to you today.

In [ ]:
!nvidia-smi

In [ ]:
!nvidia-smi topo -m 

While your work today will be on a single node, all the techniques you learn today, can be used to run your applications across clusters of multi-GPU nodes.

## Environment Modules on OGBON

These modules must be initialized before running the jupyter-notebook:
```cpp
Currently Loaded Modulefiles:
    1) anaconda3/2022.05 
    2) cuda/11.6         
    3) ucx/1.12.0-cuda-11.6-ofed-5.4
    4) gcc/11.1.0  
    5) openmpi/4.1.1-cuda-11.6-ofed-5.4
    6) intel/parallel-studio-xe/2020.2    
    7) nvhpc/22.11
    8) llvm/11.0.0
```

## Table of Contents

During the workshop today you will work through each of the following notebooks with your instructor:

- [Accelerate a Thermal Conductivity Application](2-heat.ipynb): You will begin by familiarizing yourself with a single GPU implementation of the Accelerate a Thermal Conductivity Application, which we will use to introduce  multi-resources programming paradigms.
- [Seismic Modelling - 1D Wave Equation](3-wave.ipynb): You apply your day's learnings by refactoring a single GPU 1D wave equation solver to run on supercomputing environment.
- [Final Exercise](4-finalExercise.ipynb): In this exercise you apply your concepts.

## `Matrix Multiple Benchmarks`

### ⊗ Sequential

In [ ]:
%%writefile mm.c
#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>

void fill_matrix(double *A, int n){

  for(int i = 0; i < n; i++)
    for(int j = 0; j < n; j++)
      A[i*n+j] = rand()%(10-1)*1;

}

void print_matrix(double *A, int n){

  int i,j;

  for(int i = 0; i < n; i++){
    for(int j = 0; j < n; j++)
      printf("%1.2f\t", A[i*n+j]);
    printf("\n");
  }

  printf("\n");

}

int main(int argc, char **argv){

 int n = atoi(argv[1]);
 int i, j, k;

 double  *A = (double *) malloc (sizeof(double) * n * n);
 double  *B = (double *) malloc (sizeof(double) * n * n);
 double  *C = (double *) malloc (sizeof(double) * n * n);

 fill_matrix(A,n);
 fill_matrix(B,n);

 //Measuring time
  struct timeval begin, end;
  gettimeofday(&begin, 0);

 for(i = 0; i < n; i++)
  for(j = 0; j < n; j++)
    for(k = 0; k < n; k++)
      C[i*n+j]+=A[i*n+k]*B[k*n+j];

 gettimeofday(&end, 0);
 double elapsed = (end.tv_sec - begin.tv_sec) + (end.tv_usec - begin.tv_usec)*1e-6;

 printf("%d x %d  %.3f seconds\n", n, n, elapsed);

 //print_matrix(A,n);
 //print_matrix(B,n);
 //print_matrix(C,n);

 return 0;
}

In [ ]:
!gcc mm.c -o mm

In [ ]:
!./mm 1000

### ⊗ OpenMP

In [ ]:
%%writefile mm-omp.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>
#include <sys/time.h>

void fill_matrix(double *A, int n)
{
  for(int i = 0; i < n; i++)
    for(int j = 0; j < n; j++)
      A[i*n+j] = rand()%(10-1)*1;
}

void print_matrix(double *A, int n)
{
  for(int i = 0; i < n; i++){
    for(int j = 0; j < n; j++)
      printf("%1.2f\t", A[i*n+j]);
   printf("\n");
  }

  printf("\n");
}

int main(int argc, char **argv)
{
  int n = atoi(argv[1]);  
  int i, j, k;
  struct timeval begin, end;
  
  double  *A = (double *) malloc(sizeof(double) * n * n);
  double  *B = (double *) malloc(sizeof(double) * n * n);
  double  *C = (double *) malloc(sizeof(double) * n * n);

  fill_matrix(A,n);
  fill_matrix(B,n);

  gettimeofday(&begin, 0);
     
  #pragma omp parallel for private(i,j,k)
   for(i = 0; i < n; i++) 
    for(j = 0; j < n; j++)
      for(k = 0; k < n; k++) 
        C[i*n+j] += A[i*n+k] * B[k*n+j];
    
   gettimeofday(&end, 0);
  
   long seconds = end.tv_sec - begin.tv_sec;
   long microseconds = end.tv_usec - begin.tv_usec;
   double elapsed = seconds + microseconds*1e-6;
    
   printf("%d x %d  %.2f seconds\n", n, n, elapsed);    
    
   //print_matrix(A,n);
   //print_matrix(B,n);
   //print_matrix(C,n);

   return 0;
}

In [ ]:
!gcc mm-omp.c -o mm-omp -fopenmp -O3

In [ ]:
!OMP_NUM_THREADS=1 ./mm-omp 1000

### ⊗ CUDA

In [3]:
%%writefile mm-CUDA.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <sys/time.h>

__global__ void kernel(double *A, double *B, double *C, int n) 
{  
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int j = blockIdx.y * blockDim.y + threadIdx.y;

  if(i < n && j < n)
    for( int k = 0; k < n; k++) 
       C[i*n+j] += A[i*n+k] * B[k*n+j];

}
 
void mult_matrix_cpu(double *A, double *B, double *C, int n) 
{
   for(int i = 0; i < n; i++) 
      for(int j = 0; j < n; j++)
         for(int k = 0; k < n; k++) 
            C[i*n+j]+=A[i*n+k]*B[k*n+j];
        
}

void fill_matrix(double *A, int n)
{ 
   for(int i=0; i < n; i++)
     for(int j=0; j < n; j++)
       A[i*n+j] = rand()%(10-1)*1;
   
}

void print_matrix(double *A, int n)
{
  for(int i = 0; i < n; i++){
    for(int j = 0; j < n; j++)
      printf("%1.2f\t", A[i*n+j]);
    printf("\n");
  }

  printf("\n");

}

int main(int argc, char **argv)
{
    int n = atoi(argv[1]);
    int sizeblock = atoi(argv[2]);
    struct timeval begin, end;

    /*Host*/
    double *A_host=(double *) malloc (n * n * sizeof(double));
    double *B_host=(double *) malloc (n * n * sizeof(double));
    double *C_host=(double *) malloc (n * n * sizeof(double));
        
    fill_matrix(A_host,n);
    fill_matrix(B_host,n);
      
    //print_matrix(A_host,n);
    //print_matrix(B_host,n);

    gettimeofday(&begin, 0);
    
    /*Device*/
    double *A_device;
    double *B_device;
    double *C_device;

    cudaMalloc((void**)&A_device, n * n * sizeof(double) ); 
    cudaMalloc((void**)&B_device, n * n * sizeof(double) ); 
    cudaMalloc((void**)&C_device, n * n * sizeof(double) ); 

    cudaMemcpy(A_device, A_host, n * n * sizeof(double), cudaMemcpyHostToDevice ); 
    cudaMemcpy(B_device, B_host, n * n * sizeof(double), cudaMemcpyHostToDevice ); 

    /*Computational GRID: (Grid: 2D Block: 2D)*/
    dim3 NUMBER_OF_BLOCKS ( (int) ceil( (float) n / sizeblock), (int) ceil( (float)n / sizeblock) );
    dim3 NUMBER_OF_THREADS( sizeblock, sizeblock);  

          kernel<<< NUMBER_OF_BLOCKS, NUMBER_OF_THREADS >>>(A_device, B_device, C_device, n);      

    cudaMemcpy(C_host, C_device, n * n * sizeof(double), cudaMemcpyDeviceToHost ); 

    //print_matrix(C_host, n );

    gettimeofday(&end, 0);
    
    long seconds = end.tv_sec - begin.tv_sec;
    long microseconds = end.tv_usec - begin.tv_usec;
    double elapsed = seconds + microseconds*1e-6;
    
    printf("%d x %d  %.3f seconds\n", n, n, elapsed);  
    
    cudaFree(A_device );
    cudaFree(B_device );
    cudaFree(C_device );
  
    return 0;
}

Overwriting mm-CUDA.cu


In [ ]:
!nvcc mm-CUDA.cu -o mm-CUDA

In [ ]:
!./mm-CUDA 1000 64

### ⊗ OpenACC

In [ ]:
%%writefile mm-openacc.c
#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>

void fill_matrix(double *A, int n)
{
  for(int i = 0; i < n; i++)
    for(int j = 0; j < n; j++)
      A[i*n+j] = rand()%(10-1)*1; 
}

void print_matrix(double *A, int n)
{
  for(int i = 0; i < n; i++){
    for(int j = 0; j < n; j++)
      printf("%1.2f\t", A[i*n+j]);
    printf("\n");
  }

  printf("\n");

}

int main(int argc, char **argv)
{
  int n = atoi(argv[1]);  
  int i, j, k;
  struct timeval begin, end;
 
  double *A = (double *) malloc (sizeof(double) * n * n);
  double *B = (double *) malloc (sizeof(double) * n * n);
  double *C = (double *) malloc (sizeof(double) * n * n);

  fill_matrix(A,n);
  fill_matrix(B,n);
 
  gettimeofday(&begin, 0);
      
  #pragma acc data present_or_copyin(A[:n*n], B[:n*n], n) copyout(C[:n*n])
   #pragma acc parallel 
    #pragma acc loop
     for(i = 0; i < n; i++) 
       for(j = 0; j < n; j++)
         for(k = 0; k < n; k++) 
           C[i*n+j] += A[i*n+k] * B[k*n+j];

   gettimeofday(&end, 0); 
  
   long seconds = end.tv_sec - begin.tv_sec;
   long microseconds = end.tv_usec - begin.tv_usec;
   double elapsed = seconds + microseconds*1e-6;
    
    printf("%d x %d  %.2f seconds\n", n, n, elapsed);  
     
  //print_matrix(A,n);
  //print_matrix(B,n);
  //print_matrix(C,n);

  return 0;
}

In [ ]:
!pgcc mm-openacc.c -o mm-openacc -acc

In [ ]:
!./mm-openacc 1000

### ⊗ OpenMP5

In [ ]:
%%writefile mm-omp5.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>
#include <sys/time.h>

void fill_matrix(double *A, int n)
{ 
  for(int i = 0; i < n; i++)
    for(int j = 0; j < n; j++)
      A[i*n+j] = rand()%(10-1)*1; 
}

void print_matrix(double *A, int n)
{
  for(int i = 0; i < n; i++){
    for(int j = 0; j < n; j++)
      printf("%1.2f\t", A[i*n+j]);
    printf("\n");
  }
  
  printf("\n");
}

int main(int argc, char **argv)
{
  int n = atoi(argv[1]);  
  int i, j, k;
  struct timeval begin, end;

  double  *A = (double *) malloc (sizeof(double) * n * n);
  double  *B = (double *) malloc (sizeof(double) * n * n);
  double  *C = (double *) malloc (sizeof(double) * n * n);

  fill_matrix(A,n);
  fill_matrix(B,n);

  gettimeofday(&begin, 0);
    
  #pragma omp target data map(to:A[:n*n], B[:n*n], n) map(from:C[:n*n])
  {
   #pragma omp target teams distribute parallel for private(i,j,k)
   for(i = 0; i < n; i++) 
     for(j = 0; j < n; j++)
       for(k = 0; k < n; k++) 
         C[i*n+j] += A[i*n+k] * B[k*n+j];
  }

   gettimeofday(&end, 0); 
    
   long seconds = end.tv_sec - begin.tv_sec;
   long microseconds = end.tv_usec - begin.tv_usec;
   double elapsed = seconds + microseconds*1e-6;
    
    printf("%d x %d  %.2f seconds\n", n, n, elapsed);   
  
  //print_matrix(A,n);
  //print_matrix(B,n);
  //print_matrix(C,n);

  return 0;
}

In [ ]:
!clang mm-omp5.c -o mm-omp5 -fopenmp -fopenmp-targets=nvptx64-nvidia-cuda

In [ ]:
!./mm-omp5 1000

### Comparison Performance Analysis using 1-GPU

| Program Version      | Execution Time (sec.)  | Speedup      |
| :---                 |    :----:              |        ---:  |
| Serial               | 4.55                   | 1X           |
| OpenMP T=36          | 0.05                   | 91X          |
| OpenACC              | 0.64                   | 7X           | 
| CUDA                 | 0.19                   | 24X          | 
| OpenMP5              | 2.11                   | 2X           | 

### ⊗ `CUDA Multi-GPU`

### Strategy: Static and Syncronous Partition

In [1]:
%%writefile mm-multi-GPU-sync.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>

// CUDA kernel to perform matrix multiplication
__global__ void matrixMultiply(float *A, float *B, float *C, int n) {
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int j = blockIdx.y * blockDim.y + threadIdx.y;

  if(i < n && j < n)
    for( int k = 0; k < n; k++) 
       C[i*n+j] += A[i*n+k] * B[k*n+j];

}

// Function to perform matrix multiplication synchronously on multiple GPUs
void matrixMultiplySync(float *a, float *b, float *c, int n, int num_gpus) {
    
    float *d_a[num_gpus], *d_b[num_gpus], *d_c[num_gpus];

    int block_size = 2;
    dim3 threadsPerBlock(block_size, block_size);
    dim3 numBlocks((n + block_size - 1) / block_size, (n + block_size - 1) / block_size);

    for (int i = 0; i < num_gpus; ++i) {
        cudaSetDevice(i);
        cudaMalloc(&d_a[i], n * n * sizeof(float));
        cudaMalloc(&d_b[i], n * n * sizeof(float));
        cudaMalloc(&d_c[i], n * n * sizeof(float));
        cudaMemcpy(d_a[i], a, n * n * sizeof(float), cudaMemcpyHostToDevice);
        cudaMemcpy(d_b[i], b, n * n * sizeof(float), cudaMemcpyHostToDevice);
        
                 matrixMultiply<<<numBlocks, threadsPerBlock>>>(d_a[i], d_b[i], d_c[i], n);
        
        cudaMemcpy(c, d_c[i], n * n * sizeof(float), cudaMemcpyDeviceToHost);
    }

    for (int i = 0; i < num_gpus; ++i) {
        cudaSetDevice(i);
        cudaFree(d_a[i]);
        cudaFree(d_b[i]);
        cudaFree(d_c[i]);
    }
}

void printMatrix(float *A, int n)
{
  for(int i = 0; i < n; i++){
    for(int j = 0; j < n; j++)
      printf("%1.2f\t", A[i*n+j]);
    printf("\n");
  }

  printf("\n");

}

void fillMatrix(float *A, int n)
{ 
   for(int i=0; i < n; i++)
     for(int j=0; j < n; j++)
       A[i*n+j] = 1;//rand()%(10-1)*1;
   
}

int main(int argc, char **argv) {

    int n = atoi(argv[1]);         // Matrix size
    int num_gpus = atoi(argv[2]);  // Number of GPUs

    float *a = (float*)malloc(n * n * sizeof(float));
    float *b = (float*)malloc(n * n * sizeof(float));
    float *c = (float*)malloc(n * n * sizeof(float));

    // Initialize matrices 'a' and 'b' here
    fillMatrix(a, n);
    fillMatrix(b, n);
    
    // Call matrix multiplication a/synchronously on multiple GPUs
    matrixMultiplySync(a, b, c, n, num_gpus);
    
    // Print or use the result matrix 'c' as needed
    printMatrix(a, n);
    printMatrix(b, n);
    printMatrix(c, n);

    free(a);
    free(b);
    free(c);

    return 0;
}

Writing mm-multi-GPU-sync.cu


In [ ]:
 !nvcc mm-multi-GPU-sync.cu -o mm-multi-GPU-sync

In [ ]:
!./mm-multi-GPU-sync 16 2

### Strategy: Static and asynchronous Partition using OpenMP

In [ ]:
%%writefile mm-multi-GPU-async.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <omp.h>

// CUDA kernel to perform matrix multiplication
__global__ void matrixMultiply(float *A, float *B, float *C, int n) {
  
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int j = blockIdx.y * blockDim.y + threadIdx.y;

  if(i < n && j < n)
    for( int k = 0; k < n; k++) 
       C[i*n+j] += A[i*n+k] * B[k*n+j];
}

void matrixMultiplyAsync(float *a, float *b, float *c, int n, int num_gpus) {
 
    #pragma omp parallel num_threads(num_gpus)
    {
        int gpu_id = omp_get_thread_num();
        cudaSetDevice(gpu_id);

        float *d_a, *d_b, *d_c;
        cudaMalloc(&d_a, n * n * sizeof(float));
        cudaMalloc(&d_b, n * n * sizeof(float));
        cudaMalloc(&d_c, n * n * sizeof(float));

        cudaMemcpyAsync(d_a, a, n * n * sizeof(float), cudaMemcpyHostToDevice);
        cudaMemcpyAsync(d_b, b, n * n * sizeof(float), cudaMemcpyHostToDevice);

        //dim3 threadsPerBlock(2, 2);
        //dim3 numBlocks((n + 15) / 16, (n + 15) / 16);
        int block_size = 2;
        dim3 threadsPerBlock(block_size, block_size);
        dim3 numBlocks((n + block_size - 1) / block_size, (n + block_size - 1) / block_size);

        matrixMultiply<<<numBlocks, threadsPerBlock>>>(d_a, d_b, d_c, n);

        cudaMemcpyAsync(c, d_c, n * n * sizeof(float), cudaMemcpyDeviceToHost);

        cudaFree(d_a);
        cudaFree(d_b);
        cudaFree(d_c);
    }
}

void printMatrix(float *A, int n)
{
  for(int i = 0; i < n; i++){
    for(int j = 0; j < n; j++)
      printf("%1.2f\t", A[i*n+j]);
    printf("\n");
  }

  printf("\n");

}

void fillMatrix(float *A, int n)
{ 
   for(int i=0; i < n; i++)
     for(int j=0; j < n; j++)
       A[i*n+j] = 1;//rand()%(10-1)*1;  
}


int main(int argc, char **argv) {

    int n = atoi(argv[1]);         // Matrix size
    int num_gpus = atoi(argv[2]);  // Number of GPUs

    float *a = (float*)malloc(n * n * sizeof(float));
    float *b = (float*)malloc(n * n * sizeof(float));
    float *c = (float*)malloc(n * n * sizeof(float));

    // Initialize matrices 'a' and 'b' here
    fillMatrix(a, n);
    fillMatrix(b, n);
    
    // Call matrix multiplication asynchronously on multiple GPUs 
    matrixMultiplyAsync(a, b, c, n, num_gpus);

    // Print or use the result matrix 'c' as needed
    printMatrix(a, n);
    printMatrix(b, n);
    printMatrix(c, n);

    free(a);
    free(b);
    free(c);

    return 0;
}

In [ ]:
 !nvcc mm-multi-GPU-async.cu -o mm-multi-GPU-async -Xcompiler -fopenmp

In [ ]:
!./mm-multi-GPU-async 16 2

## Next

Please continue to the next notebook: Please continue to the next notebook: [Accelerate a Thermal Conductivity Application](2-heat.ipynb).